
<h1 style="font-family:verdana;font-size:300%;text-align:center;background-color:#f2f2f2;color:#0d0d0d">AMMI NLP - Review sessions</h1>

<h1 style="font-family:verdana;font-size:180%;text-align:Center;color:#993333"> Lab 3: n-gram models </h1>


[![Open In Colab <](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1mA8WXHWKzgaEHObVAJS-Iiu6EzNbeimE?usp=sharing)

**Big thanks to Amr Khalifa who improved this lab and made it to a Jupyter Notebook!**

In [1]:
import io, sys, math, re
from collections import defaultdict
import numpy as np

In [2]:
# data_loader
def load_data(filename):
    '''
    parameters:
    filename (string): datafile
    
    Returns:
    data (list of lists): each list is a sentence of the text 
    vocab (dictionary): {word: no of times it appears in the text}
    '''
    fin = io.open(filename, 'r', encoding='utf-8')
    data = []
    vocab = defaultdict(lambda:0)
    for line in fin:
        sentence = line.split()
        data.append(sentence)
        for word in sentence:
            vocab[word] += 1
    return data, vocab

In [3]:

print("load training set..")
print("\n")
train_data, vocab = load_data("/content/train1.txt")
print(train_data[0])
print("\n")
print("how :",vocab['how'])
print("load validation set")
valid_data, _ = load_data("/content/valid1.txt")


load training set..


['<s>', 'my', 'fathers', "don't", 'speak', 'dutch.', '</s>']


how : 107
load validation set


In [4]:
def remove_rare_words(data, vocab, mincount = 1):
    '''
    Parameters:
    data (list of lists): each list is a sentence of the text 
    vocab (dictionary): {word: no of times it appears in the text}
    mincount(int): the minimum count 
    
    Returns: 
    data_with_unk(list of lists): data after replacing rare words with <unk> token
    '''
    # replace words in data that are not in the vocab 
    # or have a count that is below mincount

    ## FILL CODE
    # data_with_unk = [w if w in vocab[w] > mincount else 'unk' for w in data]

    data_with_unk=[]
    for text in data:
      for i in range(len(text)):
        if vocab[text[i]]<= mincount:
          text[i]='unk'

      data_with_unk.append(text)
    return data_with_unk

In [5]:
print("remove rare words")
train_data = remove_rare_words(train_data, vocab, mincount = 1)
valid_data = remove_rare_words(valid_data, vocab, mincount = 1)
#train_data
print(train_data[0])


remove rare words
['<s>', 'my', 'unk', "don't", 'speak', 'unk', '</s>']


In [6]:
# print(train_data[0:2])

In [7]:
def build_ngram(data, n):
    '''
    Parameters:
    data (list of lists): each list is a sentence of the text 
    n (int): size of the n-gram
    
    Returns:
    prob (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    '''
    counts = defaultdict(lambda: defaultdict(lambda: 0.0))

    total_number_words=defaultdict(lambda: 0)
    # total_number_words=0

    for sentence in data: #data=[ [] ]
        sentence = tuple(sentence)
        ## FILL CODE
        # dict can be indexed by tuples
        # store in the same dict all the ngrams
        # by using the context as a key and the word as a value
        for j in range(n+1):
          for i in range(len(sentence)-j):

            #the total of number of the words begining of the bigram forexample 
            #example sentence =(there, is, the, cat)
            #{'there': 9}

            # total_number_words[sentence[i]]+=1
            n_gram=sentence[i:i+j] #example sentence =(there, is, the, cat)
            counts[n_gram][sentence[i+j]]+=1  #count={'{there,is}','{is:5}'} 

    # <s>we sat in the house
    # <s>we sat here we two and we said 
    # <s>how we wish we had something to do

    # Extract some probabilities: 
    # P(sat | we) = 2/6 = 0.33, P(wish | we) = 0.17, P(in | sat) = 0.5
              

    prob = defaultdict(lambda: defaultdict(lambda: 0.0))
    # Build the probabilities from the counts
    # Be careful with how you normalize!

    for context in counts.keys():
      ## FILL CODE
      total_count = 0
      for word in counts[context]:
        total_count += counts[context][word]
      for word in counts[context]:
        prob[context][word] = (counts[context][word]/total_count)
      
    return prob

In [8]:
# test_data = train_data[:1]
# print(test_data)
# print("")
# build_ngram(test_data, n=2)

In [9]:
# RUN TO BUILD NGRAM MODEL
n = 3
print("build ngram model with n = ", n)
model = build_ngram(train_data, n)
# model

build ngram model with n =  3


Here, implement a recursive function over shorter and shorter context to compute a "stupid backoff model". An interpolation model can also be implemented this way.

In [10]:
def get_prob(model, context, w):
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    } 
    context (list of strings): a sentence
    w(string): the word we need to find it's probability given the context
    
    Retunrs:
    prob(float): probability of this word given the context 
    '''

    # code a recursive function over 
    # smaller and smaller context
    # to compute the backoff model
    
    ## FILL CODE

    if context in model and w in model[context]:
        return model[context][w]
    else:
        return 0.4*get_prob(model, context[1:], w)


In [16]:
# context=('<s>', 'my', 'unk', "don't", 'speak', 'unk', '</s>')
# print(get_prob(model, context, 'speak'))

2.979495961325716e-07


In [14]:
def perplexity(model, data, n):
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    } 
    data (list of lists): each list is a sentence of the text
    n(int): size of the n-gram
    
    Retunrs:
    perp(float): the perplexity of the model 
    '''
    ## FILL CODE
    perp = 0.0
    N=0
    for sentence in data:
        sentence = tuple(sentence)
        for i in range(1, len(sentence)):
            val = min(n-1, i)
            context = sentence[i-val:i]
            #logarithm of the perplexity
            perp += -math.log(get_prob(model, context, sentence[i]))
            N += 1
    perp = math.exp(perp/N)
    return perp

In [15]:
# COMPUTE PERPLEXITY ON VALIDATION SET
print("The perplexity is", perplexity(model, valid_data, n=3))

The perplexity is 62.065880784180806


In [17]:
def get_proba_distrib(model, context):
    ## need to get the the words after the context and their probability of appearance
    ## after this context 
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    context (list of strings): the sentence we need to find the words after it and 
    thier probabilites
    
    Retunrs:
    words_and_probs(dic): {word: probability of word given context}
    
    '''
    # code a recursive function over context
    # to find the longest available ngram
    
    ## FILL CODE
    
    if context in model:
        return model[context]
    else:
        return get_proba_distrib(model, context[1:])

In [19]:
# context1=('<s>', 'my', 'unk', "don't", 'speak', 'unk', '</s>')
# print(get_proba_distrib(model, context1))

defaultdict(<function build_ngram.<locals>.<lambda>.<locals>.<lambda> at 0x7ff54c37b950>, {'<s>': 0.08427417521633049, 'my': 0.00307636352614909, 'unk': 0.08683528573809993, "don't": 0.003015745525633837, 'speak': 0.000181854001545759, '</s>': 0.08427417521633049, 'tomorrow': 6.0618000515253e-05, 'is': 0.01113855759467774, 'the': 0.07075636110142906, 'election.': 4.546350038643975e-05, 'he': 0.006516435055389698, 'worked': 7.577250064406626e-05, 'hard': 0.000242472002061012, 'to': 0.026687074726840135, 'make': 0.000666798005667783, 'his': 0.00372800703168806, 'ideas': 9.09270007728795e-05, 'understandable': 3.03090002576265e-05, 'at': 0.003909861033233819, 'meeting.': 0.000181854001545759, 'tom': 0.022398351190385985, 'going': 0.001576068013396578, 'stay': 0.0003940170033491445, 'in': 0.013639050115931925, 'australia': 0.0003182445027050783, 'until': 0.000363708003091518, 'end': 0.0003485535029627048, 'of': 0.014851410126236987, 'john': 0.0010911240092745542, 'took': 0.0005455620046372

In [20]:
def generate(model):
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    
    Retunrs:
    sentence (list of strings): a sentence sampled according to the language model. 
    

    '''
    # generate a sentence. A sentence starts with a <s> and ends with a </s>
    # Possiblly a use function is:
    # np.random.choice(x, 1, p = y)

    # where x is a list of things to sample from
    # and y is a list of probability (of the same length as x)
    sentence = ["<s>"]
    while sentence[-1] != "</s>" and len(sentence)<100:
        ## FILL CODE
        proba = get_proba_distrib(model, tuple(sentence))
        w = np.random.choice((list(proba.keys())), 1, p = list(proba.values()))
        sentence.append(w[0])
    return sentence

In [23]:
# GENERATE A SENTENCE FROM THE MODEL
print("Generated sentence: ",generate(model))

Generated sentence:  ['<s>', 'they', 'said', 'that', 'they', 'had', 'to', 'fight', 'for', 'their', 'unk', '</s>']


Once you are done implementing the model, evaluation and generation code, you can try changing the value of `n`, and play with a larger training set (`train2.txt` and `valid2.txt`). You can also try to implement an interpolation model.